In [14]:
import requests
from bs4 import BeautifulSoup
import pickle
import re
import pandas as pd

In [8]:
def num_pages(link):
    n = 1 + len(BeautifulSoup(requests.get(link).text, "lxml").findAll('a', class_ = 'post-page-numbers'))
    return n

In [ ]:
!git clone https://github.com/featztex/PsychoBot

In [10]:
links = []
with open('PsychoBot/parsing/links.pickle', 'rb') as f:
    links = pickle.load(f)

In [16]:
phrases = []
authors = []

for link in links:

    for i in range(1, num_pages(link) + 1):

        if i == 1:
            page_url = link
        else:
            page_url = link + '/' + str(i)
        
        page = requests.get(page_url)

        soup = BeautifulSoup(page.text, "lxml").findAll('p')[:-2]
        
        #костыль moment
        if link == 'https://uaforizm.com/vyskazyvanija-zhizni.html':
            soup = soup[1::]

        cnt = 0
        for j in soup:
            string = j.text
            string = re.sub(r"[\d]+ | [a-zA-Z]+", r"", string).strip()
            if cnt % 2 == 0:
                phrases.append(string)
            else:
                authors.append(string)
            cnt += 1

In [21]:
ph = pd.Series(phrases)
auth = pd.Series(authors)
data = pd.DataFrame({'Цитата' : ph, 'Автор' : auth})

In [22]:
data.to_csv('data.csv')